# 测试训练好的模型的决策效果

In [1]:
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO,DDPG,DQN,A2C
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import gym
import numpy as np
import time
from stable_baselines3.common.callbacks import EvalCallback
import env
env = env.ElectricVehicleEnv()
#vec_env = make_vec_env(env, n_envs=4)
#check_env(env)
import numpy as np
from env import ElectricVehicleEnv
from stable_baselines3 import A2C
# 加载训练好的模型
#model = A2C.load("best_model/A2C/best_model.zip")
#model = DQN.load("best_model/DQN/best_model.zip")
#model = A2C.load("model/a2c_electric_vehicle.zip")
model = DQN.load("model/dqn_retrained_electric_vehicle3.zip")

# 测试模型

In [2]:
import logging
import os
# 确保 'result' 目录存在
result_dir = os.path.join(os.getcwd(), 'result')
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# 设置日志文件的路径，确保日志文件生成在 'result' 目录中
log_file_path = os.path.join(result_dir, 'model_output.log')

# 配置日志，确保写入文件
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(message)s', filemode='w')

def log_info(*args):
    """
    替代print函数，用logging.info来保存日志。
    """
    message = " ".join(map(str, args))
    print(message)
    logging.info(message)

def print_step_details(info, state, reward, done):
    """
    打印步骤中需要的详细信息（从节点、当前电池电量、剩余时间、Done, 奖励）。
    """
    to_node = info.get('to_node')  # 确保info中有from_node字段
    battery = state[-1]  # 剩余电池电量在state的最后一个位置
    remaining_time = state[-2] / 50 # 剩余时间在state倒数第二个位置
    log_info(f"  到节点 (From Node): {to_node}")
    log_info(f"  当前电池电量 (Battery): {battery}")
    log_info(f"  剩余时间 (Remaining Time): {remaining_time}")
    log_info(f"  奖励 (Reward): {reward}")
    log_info(f"  Done: {done}")

# 初始化一些参数
num_episodes = 50  # 执行100个回合
log_info("Initial State:")
episode = 0

# 执行100个回合
for episode in range(num_episodes):
    # 每个回合开始时重置环境
    obs, info = env.reset(index=episode)
    done = False
    step = 0

    log_info(f"\nEpisode {episode + 1}:")
    log_info("-" * 30)
    log_info(f"\nStep {step}:")
    print_step_details(info, obs, 0, done)
    
    # 在一个回合中执行动作，直到该回合结束
    while not done:
        # 使用训练好的模型预测动作
        action, _states = model.predict(obs, deterministic=True)
        # 执行动作
        obs, reward, done, truncated, info = env.step(action)

        log_info("-" * 30)
        log_info(f"\nStep {step + 1}:")
        step += 1
        print_step_details(info, obs, reward, done)

        # 如果当前回合结束，跳出while循环，开始下一个回合
        if done:
            log_info(f"Episode {episode + 1} finished! Resetting environment for next episode.")
            break

# 确保日志及时写入文件
logging.shutdown()


Initial State:

Episode 1:
------------------------------

Step 0:
  到节点 (From Node): 26
  当前电池电量 (Battery): 34.0
  剩余时间 (Remaining Time): 1.7917361450195313
  奖励 (Reward): 0
  Done: False
------------------------------

Step 1:
  到节点 (From Node): 48
  当前电池电量 (Battery): 44.85390090942383
  剩余时间 (Remaining Time): 1.678580322265625
  奖励 (Reward): 5.878399064442988
  Done: False
------------------------------

Step 2:
  到节点 (From Node): 40
  当前电池电量 (Battery): 72.84883117675781
  剩余时间 (Remaining Time): 1.3830258178710937
  奖励 (Reward): 200.98487242265045
  Done: True
Episode 1 finished! Resetting environment for next episode.

Episode 2:
------------------------------

Step 0:
  到节点 (From Node): 28
  当前电池电量 (Battery): 47.0
  剩余时间 (Remaining Time): 1.934728546142578
  奖励 (Reward): 0
  Done: False
------------------------------

Step 1:
  到节点 (From Node): 3
  当前电池电量 (Battery): 51.5717658996582
  剩余时间 (Remaining Time): 1.8865133666992187
  奖励 (Reward): -57.21956044676499
  Done: False
-------

In [4]:
import pandas as pd

def extract_data_from_log(log_file):
    """
    从日志文件中提取每个车辆的路径、剩余电量和剩余截止时间信息。
    """
    car_data = []
    current_car = []
    
    with open(log_file, 'r', encoding='GB2312') as file:
        for line in file:
            if "节点 (From Node)" in line:
                from_node = int(line.split(":")[1].strip())
            elif "当前电池电量 (Battery)" in line:
                battery = float(line.split(":")[1].strip())
            elif "剩余时间 (Remaining Time)" in line:
                remaining_time = float(line.split(":")[1].strip())
            elif "Done" in line:
                done = "True" in line
                
                # 添加当前步骤的数据
                current_car.append([from_node, battery, remaining_time])
                
                # 如果done为True，代表一辆车的数据结束，保存该车的数据
                if done:
                    car_data.append(current_car)
                    current_car = []
                    
    return car_data

def create_excel_from_data(car_data, output_file):
    """
    将提取的数据转换为 Excel 文件，每辆车的数据作为一个单独的表格。
    """
    with pd.ExcelWriter(output_file) as writer:
        for i, car_steps in enumerate(car_data, 1):
            # 创建DataFrame并添加到Excel中
            df = pd.DataFrame(car_steps, columns=['路径（点集）', '剩余电量（kwh）', '剩余截止时间（hour）'])
            df.index = range(1, len(df) + 1)
            df.to_excel(writer, sheet_name=f'车辆_{i}', index=False)

def main(log_file_path, output_excel_path):
    # 从日志文件中提取数据
    car_data = extract_data_from_log(log_file_path)
    
    # 创建Excel文件
    create_excel_from_data(car_data, output_excel_path)
    print(f"文件已保存到: {output_excel_path}")

# 使用示例
log_file = 'result/model_output.log'  # 替换为你的日志文件路径
output_excel = 'result/electric_vehicles.xlsx'  # 替换为你想要保存的Excel文件路径

main(log_file, output_excel)

文件已保存到: result/electric_vehicles.xlsx
